In [4]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, confusion_matrix, ConfusionMatrixDisplay, fowlkes_mallows_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime
import math

In [5]:
# Read in CSV Data
dataset = pd.read_csv('./data/creditcard.csv')
display(dataset.head())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
"""
Utility functions for
1. Splitting data into training and test
2. Creating a Gaussian Mixture Model and Running the EM Algorithm
3. Performing K-Fold Cross Validation with 
"""
### Util functions to train models ###
def get_train_test_splits(X: np.ndarray, Y: np.ndarray, train_index: np.ndarray, test_index: np.ndarray) -> tuple:
    """
    Splits data and labels into train and test portions

    Args:
        X (np.ndarray): Normalized Data
        Y (np.ndarray): Labels
        train_index (np.ndarray): Indices corresponding to train data/labels from K-Fold
        test_index (np.ndarray): Indices corresponding to test data/labels from K-Fold

    Returns:
        tuple: Returns x_train, x_test, y_train, y_test
    """
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    return x_train, x_test, y_train, y_test

def get_trained_model(x_train: np.ndarray, num_iterations: int) -> GaussianMixture:
    """
    Creates a Gaussian Mixture Model and runs the EM Algorithm

    Args:
        x_train (np.ndarray): Training Data
        num_iterations (int): Number of times to run the EM Algorithm

    Returns:
        GaussianMixture: Trained Gaussian Mixture Model
    """
    gmm = GaussianMixture(n_components=2, n_init=num_iterations)
    return gmm.fit(x_train)

def get_model_predictions(model: GaussianMixture, x_test: np.ndarray, y_test: np.ndarray) -> tuple:
    """Classifies test data based on trained Gaussian Model

    Args:
        model (GaussianMixture): The Trained Gaussian Mixture Model
        x_test (np.ndarray): Testing data
        y_test (np.ndarray): Testing Labels (the ground truth)

    Returns:
        tuple: x_test, y_test, predictions
    """
    predictions = model.predict(x_test)
    return x_test, y_test, predictions

def train_and_test_model(unsupervised_data: np.ndarray) -> list:
    """
    Runs K-Fold Validation on the entire data, training and getting predictions for each fold

    Args:
        unsupervised_data (np.ndarray): Entire data from preprocessing

    Returns:
        list: List of (x_test, y_test, predictions) for displaying results later
    """
    K_FOLDS = 5
    NUM_ITERATIONS = 10

    results = []
    count = 0
    for train_index, test_index in KFold(shuffle=True, n_splits=K_FOLDS).split(unsupervised_data):
        count += 1
        print(f'Running fold # {count}')

        # Splitting data and obtaining trained model
        print("Splitting Data")
        x_train, x_test, _, y_test = get_train_test_splits(unsupervised_data, labels, train_index, test_index)
        
        # Training Model
        print("Training Model")
        model = get_trained_model(x_train, NUM_ITERATIONS)

        # Get Test Results for Models
        print("Testing and Obtaining Results")
        print()
        test_result = get_model_predictions(model, x_test, y_test)
        results.append(test_result)
    return results

In [7]:
''' 
Utility functions for 
1. Displaying Confusion Matrices
2. Calculating recall_rates
3. Calculating accuracies
4. Calculating Silhouette Scores
5. Calculating Fowlkes Mallows Scores
6. Displaying clusterings with respect to V1 and V2
'''
### Util functions for displaying results
def get_confusion_matrices(results: list) -> list:
    """
    Generates confusion matrices given results

    Args:
        results (list): A list of (x_test, y_test, predictions) tuples

    Returns:
        list: A list of confusion matrices, returns a list because there is a
              a confusion matrix for each for the folds in K-fold
    """
    matrices_of_confusion = []
    for i, (x_test, ground_truth, prediction) in enumerate(results):
        cm = confusion_matrix(ground_truth, prediction)
        if cm[0][0] < cm[0][1]:
            # This means the clusters were flipped (0 means fraud 1 means fraud)
            # so we switch our predictions
            results[i] = (x_test, ground_truth, 1 - prediction)
            cm = confusion_matrix(ground_truth, 1 - prediction)
        matrices_of_confusion.append(cm)
    return matrices_of_confusion

def get_recall_rates(matrices_of_confusion: list) -> list:
    """
    Calculates recall rates (percentage of frauds that were detected)

    Args:
        matrices_of_confusion (list): List of Confusion Matrices

    Returns:
        list: A list of recall rates, returns a list because there is a
              a recall rate for each for the folds in K-fold
    """
    recall_rates = []
    for confusion_matrix in matrices_of_confusion:
        recall = confusion_matrix[1][1] / (np.sum(confusion_matrix[1]))
        recall_rates.append(recall)
    return recall_rates

def get_accuracies(results: list) -> list:
    """
    Calculates accuracies (how well the model distinguishes fraud from not fraud)

    Args:
        results (list): A list of (x_test, y_test, predictions) tuples

    Returns:
        list: A list of accuracy rates, returns a list because there is a
              a recall rate for each for the folds in K-fold
    """
    accuracies = []
    for _, y_test, prediction in results:
        accuracy = np.count_nonzero(y_test == prediction) / len(y_test)
        accuracies.append(accuracy)
    return accuracies

def get_silhouette_scores(results: list) -> list:
    """
    Calculates silhouette_scores measured on a scale from -1 to 1
    (how well the clusters are distinguished and how far apart they are)

    Args:
        results (list): A list of (x_test, y_test, predictions) tuples

    Returns:
        list: A list of silhouette scores, returns a list because there is a
              a recall rate for each for the folds in K-fold
    """
    scores = []
    for x_test, _, prediction in results:
        score = silhouette_score(x_test, prediction)
        scores.append(score)
    return scores

def get_fowlkes_mallows_scores(results: list) -> list:
    """
    Calculates Fowlkes Mallows Scores (similarities between 2 clusters)
    https://scikit-learn.org/stable/modules/clustering.html#fowlkes-mallows-scores

    Args:
        results (list): A list of (x_test, y_test, predictions) tuples

    Returns:
        list: A list of Fowlkes Mallows Scores, returns a list because there is a
              a recall rate for each for the folds in K-fold
    """
    scores = []
    for _, ground_truth, prediction in results:
        score = fowlkes_mallows_score(ground_truth, prediction)
        scores.append(score)
    return scores

def plot_clusterings(results: list) -> None:
    """
    Plots Clustering with respect to V1 and V2 components

    Args:
        results (list): A list of (x_test, y_test, predictions) tuples
    """
    for count, (x_test, y_test, prediction_test) in enumerate(results):
        colors = {
            0: "green",
            1: "red"
        }
        for group in np.unique(y_test):
            indexes = np.where(group == prediction_test)
            plt.scatter(
                x_test[indexes, 1],
                x_test[indexes, 2],
                c=colors[group],
                s=5,
                label=group,
            )
        plt.title(f"Visualization of Clusterings using V1 and V2 for k={count + 1}")    
        plt.legend()
        plt.show()

In [ ]:
"""
Main steps
1. Shuffle the dataset
2. Separate the labels from the data
3. Normalize the data via MinMaxScalar
4. Start from 2 columns and add components. For each component, run K-fold validation,
   keeping track of time taken to run for each component added
"""
# Shuffling the dataset
dataset = dataset.sample(frac=1).reset_index(drop=True)

# Separating the labels from the data
labels = dataset['Class']
unsupervised_data = dataset.drop(columns=['Class'])

# Normalizing by Min/Max Scalar
unsupervised_data = MinMaxScaler(feature_range=(0, 1)).fit_transform(unsupervised_data)

times = []
col_iteration_results = []
for i in range(2, len(dataset.columns)):
    desired_columns = dataset.columns[:i]
    unsupervised_data = dataset[desired_columns]
    unsupervised_data = MinMaxScaler(feature_range=(0, 1)).fit_transform(unsupervised_data)
    start = datetime.now()
    print(f'RUNNING WITH {i} COLUMNS')
    print('---------------------------')
    results = train_and_test_model(unsupervised_data)
    end = datetime.now()
    col_iteration_results.append(results)
    times.append(math.ceil((end - start).total_seconds()))
    print()


In [ ]:
# Displaying Confusion Matrices for the iteration with all components
for matrix in get_confusion_matrices(col_iteration_results[-1]):
    ConfusionMatrixDisplay(matrix).plot()

In [ ]:
# Display other metrics
accuracies, recalls = [], []

for results in col_iteration_results:
    accuracies.append(np.mean(get_accuracies(results)))
    recalls.append(np.mean(get_recall_rates(get_confusion_matrices(results))))
silhouette_scores = get_silhouette_scores(col_iteration_results[-1])
fowlkes_mallows_scores = get_fowlkes_mallows_scores(col_iteration_results[-1])

print(f'Time taken: {times}')
print(f'Overall accuracies: {accuracies}')
print(f'Overall recall rates: {recalls}')
print(f'Silhouette Scores: {silhouette_scores}')
print(f'Fowlkes Mallows Scores: {fowlkes_mallows_scores}')

In [ ]:
# Plotting Clusterings with respect to just V1 and V2 for the iteration using all components
plot_clusterings(col_iteration_results[-1])